# Primer on Decorators

In python functions are first-class objects. This means that **functions can be passed around and used as argugments**, just like any other object (string, int, float, list etc). 

In [1]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

Here, say_hello() and be_awesome() are regular functions that expect a name given as a string. The greet_bob() function however, expects a function as its argument. We can, for instance, pass it the say_hello() or the be_awesome() function:

In [2]:
greet_bob(say_hello)

'Hello Bob'

In [3]:
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'

Note that greet_bob(say_hello) refers to two functions, but in different ways: greet_bob() and say_hello. The say_hello function is named without parentheses. This means that only a reference to the function is passed. The function is not executed. The greet_bob() function, on the other hand, is written with parentheses, so it will be called as usual.

## Inner Functions

It’s possible to define functions inside other functions. Such functions are called inner functions. Here’s an example of a function with two inner functions:

In [4]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()

In [5]:
parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


We can see the order in which inner functions are defined, but it doesn't matter because if we see how they are called, `parent()` then `second_child()` then `first_child()`. If we tried to call these functions from within `parent()` we should get an error. 

In [9]:
second_child()

NameError: name 'second_child' is not defined

### Returning Functions From Functions

Python also allows you to use functions as return values. The following example returns one of the inner functions from the outer parent() function, for example.

In [10]:
def parent(num):
    def first():
        return "I am the first function"
    
    def second():
        return "I am the second function"
    
    if num == 1:
        return first
    else:
        return second

In [15]:
first = parent(1)

In [14]:
second = parent(2)

In [16]:
first()

'I am the first function'

In [17]:
second()

'I am the second function'

## Simple Decorators

In [18]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)

In [20]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


To understand what’s going on here, look back at the previous examples. We are literally just applying everything you have learned so far.

The so-called decoration happens at the following line:

In [21]:
say_whee = my_decorator(say_whee)

In [22]:
say_whee

<function __main__.my_decorator.<locals>.wrapper()>

However, wrapper() has a reference to the original say_whee() as func, and calls that function between the two calls to print().

Put simply: **decorators wrap a function, modifying its behavior.**

Before moving on, let’s have a look at a second example. Because wrapper() is a regular Python function, the way a decorator modifies a function can change dynamically. So as not to disturb your neighbors, the following example will only run the decorated code during the day:

In [23]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def say_whee():
    print("Whee!")

say_whee = not_during_the_night(say_whee)

In [24]:
say_whee()

## Syntatic Sugar!

The way you decorated say_whee() above is a little clunky. First of all, you end up typing the name say_whee three times. In addition, the decoration gets a bit hidden away below the definition of the function.

Instead, Python allows you to use decorators in a simpler way with the @ symbol, sometimes called the “pie” syntax. The following example does the exact same thing as the first decorator example:

In [25]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_whee():
    print("Whee!")


So, @my_decorator is just an easier way of saying say_whee = my_decorator(say_whee). It’s how you apply a decorator to a function.

## Reusing Decorators

Recall that a decorator is just a regular Python function. All the usual tools for easy reusability are available. Let’s move the decorator to its own module that can be used in many other functions.

Create a file called decorators.py with the following conten

In [26]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

Note: You can name your inner function whatever you want, and a generic name like wrapper() is usually okay. You’ll see a lot of decorators in this article. To keep them apart, we’ll name the inner function with the same name as the decorator but with a wrapper_ prefix.

In [31]:
@do_twice
def say_hello():
    print('HI')

In [32]:
say_hello()

HI
HI


## Decorating Functions With Arguments

In [35]:
@do_twice
def greet(name):
    print(f"hello {name}")

In [36]:
greet('Jordan')

TypeError: wrapper_do_twice() takes 0 positional arguments but 1 was given

The problem is that the inner function wrapper_do_twice() does not take any arguments, but name="World" was passed to it. You could fix this by letting wrapper_do_twice() accept one argument, but then it would not work for the say_whee() function you created earlier.

In [37]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

In [38]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


In [40]:
@do_twice
def greet(name):
    print(f"hello {name}")

In [41]:
greet('Jordan')

hello Jordan
hello Jordan


In [43]:
hi_adam = return_greeting("Adam")

Creating greeting
Creating greeting


In [44]:
print(hi_adam)

None


Oops, your decorator ate the return value from the function.

Because the do_twice_wrapper() doesn’t explicitly return a value, the call return_greeting("Adam") ended up returning None.

To fix this, you need to make sure the wrapper function returns the return value of the decorated function. Change your decorators.py file:



In [45]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs) # here we are returning the func
    return wrapper_do_twice

In [47]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [48]:
return_greeting("jordan")

Creating greeting
Creating greeting


'Hi jordan'

In [51]:
say_hello.__name__

'wrapper_do_twice'

In [52]:
help(say_hello)

Help on function wrapper_do_twice in module __main__:

wrapper_do_twice()



However, after being decorated, say_whee() has gotten very confused about its identity. It now reports being the wrapper_do_twice() inner function inside the do_twice() decorator. Although technically true, this is not very useful information.

To fix this, decorators should use the @functools.wraps decorator, which will preserve information about the original function. Update decorators.py again:

In [53]:
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [54]:
# https://realpython.com/primer-on-python-decorators/